<a href="https://colab.research.google.com/github/HeatherDriver/MathGraph/blob/main/04_Named_Entity_Recognition_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install fuzzywuzzy
! pip install intervaltree

  Preparing metadata (setup.py) ... done
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=93ed325f1d726cd175037488674d05331ab6323b77929e4f18e8162ad8c446bf
  Stored in directory: /root/.cache/pip/wheels/fa/80/8c/43488a924a046b733b64de3fac99252674c892a4c3801c0a61
Successfully built intervaltree


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline, AutoConfig, DistilBertForTokenClassification, DistilBertModel, DistilBertConfig, DistilBertPreTrainedModel
from transformers.modeling_outputs import TokenClassifierOutput
import torch
import torch.nn as nn
from google.colab import drive, userdata
import pickle
import random
import re
import time
from collections import defaultdict
from fuzzywuzzy import fuzz
from intervaltree import Interval, IntervalTree
from collections import Counter
import json
from openai import OpenAI
import pandas as pd

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files'

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Math_Graph/pickle_files


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Define nested_dict function
def nested_dict():
  return defaultdict(nested_dict)

# Define file read function
def read_pickle(dict_file):
  with open(dict_file, 'rb') as file:
    return pickle.load(file)

# Split CamelCase to s e p a r a t e d text
def split_camel_case(text):
    # Use regex to insert a space before uppercase letters preceded by lowercase letters
    return re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)

In [ ]:
# Read in dictionary with definitions
definitions = read_pickle("short_answer_dict.pkl")
all_definitions = read_pickle("final_combined_dict.pkl")

## Checking existing NER capability

In [ ]:
# Existing tags from DistilBERT
tag_checkpoint = "dslim/distilbert-NER"
tag_tokenizer = AutoTokenizer.from_pretrained(tag_checkpoint, do_lower_case=False)
tag_model = AutoModelForTokenClassification.from_pretrained(tag_checkpoint)

tags = pipeline("ner", model=tag_model, tokenizer=tag_tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/926 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
tag_tokenizer.is_fast

True

In [ ]:
# Methods built from DistilBERT NER

def idx2string(text):
    # Split on whitespace or punctuation, punctuation is used for tokens
    words = re.findall(r'\b\w+\b|[^\s\w]', text)
    my_dict = {i: word for i, word in enumerate(words)}
    return my_dict

def tokenidx2words(ner_results):
  my_dict = dict()
  for sub_dict in ner_results:
    word, index = sub_dict['word'], sub_dict['index']
    my_dict[index] = word
  return my_dict

def tokenidx2entity(ner_results):
  my_dict = dict()
  for sub_dict in ner_results:
    entity, index = sub_dict['entity'], sub_dict['index']
    my_dict[index] = entity
  return my_dict

def idx2wordpos(text, idx2string):
  my_dict = {}
  current_pos = 0
  for idx, word in idx2string.items():
    start = text.index(word, current_pos)
    end = start + len(word)
    my_dict[idx] = (start, end)
    current_pos = end
  return my_dict

def tokenidx2entityword(example2wordidx, ner_results):
  # Create an IntervalTree
  tree = IntervalTree(Interval(start, end, key) for key, (start, end) in example2wordidx.items())

  my_dict = dict()
  for sub_dict in ner_results:
    tag_start, tag_end = sub_dict['start'], sub_dict['end']
    tag_entity = sub_dict['entity']
    token_idx = sub_dict['index']

    overlapping_intervals = tree[tag_start : tag_end] # Intervals which contain the entitry recognised
    for interval in overlapping_intervals:
      my_dict[token_idx] = {'entity' : tag_entity, 'word_idx': interval.data}

  return my_dict

def baseline_tags_list(tokens, baseline_tags):
  start_entities = list(len(tokens)*'O')
  for i in baseline_tags:
    start_entities[i['index']] = i['entity']
    assert len(start_entities) == len(tokens)
  return start_entities

In [ ]:
example = "A Hankel matrix is a type of structured matrix where each ascending skew-diagonal from left to right is constant. It is characterized by its entries being functions of a single variable, forming a pattern where each element depends only on the sum of its indices. Hankel matrices are often used in signal processing and control theory for polynomial and sequence analysis."
print(f"{example}\n")
example2idx = idx2string(example)
print(f"word index: {example2idx}\n")

example2wordidx = idx2wordpos(example, example2idx)
print(f"word start-end index: {example2wordidx}\n")

ner_results = tags(example)
print(f"tags: {ner_results}\n")

# tokens = tag_tokenizer(example)['input_ids']
# print(f"bert input ids: {len(tokens)}, {tokens}\n")

# input_ids = tag_tokenizer.encode(example)
input_ids = tag_tokenizer(example)['input_ids']
print(f"input_ids: {len(input_ids)}, {input_ids}\n")

mapper_dict = tokenidx2entityword(example2wordidx, ner_results)
print(f"mapper_dict: {mapper_dict}\n")

baseline_tags = baseline_tags_list(input_ids, ner_results)
print(f"baseline_tags: {len(baseline_tags)}, {baseline_tags}\n")

A Hankel matrix is a type of structured matrix where each ascending skew-diagonal from left to right is constant. It is characterized by its entries being functions of a single variable, forming a pattern where each element depends only on the sum of its indices. Hankel matrices are often used in signal processing and control theory for polynomial and sequence analysis.

word index: {0: 'A', 1: 'Hankel', 2: 'matrix', 3: 'is', 4: 'a', 5: 'type', 6: 'of', 7: 'structured', 8: 'matrix', 9: 'where', 10: 'each', 11: 'ascending', 12: 'skew', 13: '-', 14: 'diagonal', 15: 'from', 16: 'left', 17: 'to', 18: 'right', 19: 'is', 20: 'constant', 21: '.', 22: 'It', 23: 'is', 24: 'characterized', 25: 'by', 26: 'its', 27: 'entries', 28: 'being', 29: 'functions', 30: 'of', 31: 'a', 32: 'single', 33: 'variable', 34: ',', 35: 'forming', 36: 'a', 37: 'pattern', 38: 'where', 39: 'each', 40: 'element', 41: 'depends', 42: 'only', 43: 'on', 44: 'the', 45: 'sum', 46: 'of', 47: 'its', 48: 'indices', 49: '.', 50: 

In [ ]:
# Model requires entities to be cased to recognise them

## Encoding the definitions

In [ ]:
# Dictionary with existing tags - these are to be updated to the new custom tags
dict_file_name = 'preliminary_ner_results.pkl'

try:
  with open(dict_file_name, 'rb') as file:
    preliminary_ner_results = pickle.load(file)

except FileNotFoundError:
  print(f"{dict_file_name} not found, creating {dict_file_name}...")

  preliminary_ner_results = dict()

  for key, value in definitions.items():
    text_index = idx2string(value)
    tokens = tag_tokenizer(value).tokens()
    ner_results = tags(value)
    preliminary_ner_results[key] = {
        "text" : value,
        "text_index": text_index,
        "text_start_end_index": idx2wordpos(value, text_index),
        "tokens" : tokens,
        "baseline_tags" : baseline_tags_list(tokens, ner_results),
        "input_ids" : tag_tokenizer.encode(value),
        }

  with open(dict_file_name, 'wb') as file:
    pickle.dump(preliminary_ner_results, file)

In [ ]:
# Checks
for key in preliminary_ner_results.keys():
  if len(preliminary_ner_results[key]["input_ids"]) != len(preliminary_ner_results[key]["baseline_tags"]):
    print(key)

In [ ]:
print(f"dictionary length: {len(preliminary_ner_results)}")

print(f"Pythagorean theorem:")
for k, v in preliminary_ner_results["Pythagorean theorem"].items():
  print(f"{k}:\n   {v}")

dictionary length: 5388
Pythagorean theorem:
text:
   The Pythagorean Theorem describes the relationship between the sides of a right triangle, stating that the square of the length of the hypotenuse is equal to the sum of the squares of the lengths of the other two sides. This principle is fundamental in geometry and is used to determine distances and measurements in various applications. It highlights the intrinsic connection between the angles and sides of a right triangle.
text_index:
   {0: 'The', 1: 'Pythagorean', 2: 'Theorem', 3: 'describes', 4: 'the', 5: 'relationship', 6: 'between', 7: 'the', 8: 'sides', 9: 'of', 10: 'a', 11: 'right', 12: 'triangle', 13: ',', 14: 'stating', 15: 'that', 16: 'the', 17: 'square', 18: 'of', 19: 'the', 20: 'length', 21: 'of', 22: 'the', 23: 'hypotenuse', 24: 'is', 25: 'equal', 26: 'to', 27: 'the', 28: 'sum', 29: 'of', 30: 'the', 31: 'squares', 32: 'of', 33: 'the', 34: 'lengths', 35: 'of', 36: 'the', 37: 'other', 38: 'two', 39: 'sides', 40: '.', 41:

In [ ]:
# Creating an exceptions dictionary with prepositions, conjunctions to bypass in NER
exceptions = ['and', 'or', 'but', 'nor', 'for', 'so', 'yet', 'because', 'although', 'since', 'while', 'if', 'either', 'neither', 'both', 'not', 'only', 'also', 'about', 'above', 'across', 'after',
              'against', 'along', 'among', 'around', 'at', 'before', 'behind', 'below', 'beneath', 'beside', 'between', 'beyond', 'by', 'despite', 'down', 'during', 'except', 'for', 'from', 'in',
              'inside', 'into', 'like', 'means', 'near', 'of', 'off', 'on', 'onto', 'out', 'outside', 'over', 'past', 'since', 'through', 'throughout', 'till', 'to', 'toward', 'under', 'underneath', 'until',
              'up', 'upon', 'with', 'within', 'without']

exception_input_ids = [tag_tokenizer.encode(exception)[1] for exception in exceptions]

## Encoding Named Entity Recognition Dictionary

In [ ]:
# Read in mapped dictionary with NER
all_classifications = read_pickle("all_classifications.pkl")

In [ ]:
print(f"all_classifications dictionary length: {len(all_classifications)}")
random_sample = random.sample(list(all_classifications.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n {value}")

all_classifications dictionary length: 21696
Artinian Ring:
 {'text_index': {0: 'Artinian', 1: 'Ring'}, 'tokens': ['[CLS]', 'Art', '##inian', 'Ring', '[SEP]'], 'input_ids': [101, 2051, 17054, 8467, 102], 'ner_mapper': [2051, 8467], 'ner_tags': ['O', 'B-ALGEBRA', 'O', 'I-ALGEBRA', 'O']}
Frobenius Number:
 {'text_index': {0: 'Frobenius', 1: 'Number'}, 'tokens': ['[CLS]', 'Fr', '##obe', '##nius', 'Number', '[SEP]'], 'input_ids': [101, 13359, 21367, 19206, 7421, 102], 'ner_mapper': [13359, 7421], 'ner_tags': ['O', 'B-NUMBER-THEORY', 'O', 'O', 'I-NUMBER-THEORY', 'O']}
Finite Mathematics:
 {'text_index': {0: 'Finite', 1: 'Mathematics'}, 'tokens': ['[CLS]', 'Fin', '##ite', 'Mathematics', '[SEP]'], 'input_ids': [101, 19140, 3150, 9833, 102], 'ner_mapper': [19140, 9833], 'ner_tags': ['O', 'B-DISCRETE-MATHEMATICS', 'O', 'I-DISCRETE-MATHEMATICS', 'O']}


In [ ]:
# Update tags to ignore exceptions such as prepositions and conjunctions
for key, sub_dict in all_classifications.items():
  input_ids = sub_dict['input_ids']
  ner_tags = sub_dict['ner_tags']
  for i in range(len(input_ids)):
    if input_ids[i] in exception_input_ids:
      ner_tags[i] = 'O'
  sub_dict['ner_tags'] = ner_tags

In [ ]:
# Trim off input_ids 101 and 102 as these are the start and stop ids
input_id_mapping = dict()

for sub_dict in all_classifications.values():
  key = tuple(sub_dict['input_ids'][1:-1])
  value = sub_dict['ner_tags'][1:-1]
  assert len(key) == len(value), 'Error'
  input_id_mapping.update({key : value})

In [ ]:
# Can also match on fuzzy-matched keys between the two eg "pythagoras's theorem" and "Pythagorean theorem"
dict_file_name = 'fuzzy_key_match_results.pkl'

try:
  with open(dict_file_name, 'rb') as file:
    fuzz_dict = pickle.load(file)

except FileNotFoundError:
  print(f"{dict_file_name} not found, creating {dict_file_name}...")

  fuzz_dict = dict()
  all_classifications_keys = list(all_classifications.keys())

  for key in preliminary_ner_results.keys():
    my_dict = dict()
    for i in all_classifications_keys:
      fuzz_ratio = fuzz.ratio(key, i)
      if fuzz_ratio > 80 and fuzz_ratio < 100:
        my_dict.update({i : fuzz_ratio})
    fuzz_dict.update({key : my_dict})

  with open(dict_file_name, 'wb') as file:
    pickle.dump(fuzz_dict, file)

In [ ]:
print(f"fuzzy_key_match_results dictionary length: {len(fuzz_dict)}")
random_sample = random.sample(list(fuzz_dict.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n {value}")

fuzzy_key_match_results dictionary length: 5388
Periodic Sequence:
 {'Perrin Sequence': 81, 'Melodic Sequence': 85, 'periodic sequence': 88}
Signature Sequence:
 {'signature sequence': 89}
Euler's Totient Theorem:
 {"euler's totient theorem": 87, "Euler'S Totient Theorem": 96}


In [ ]:
# To increase the training dictionary size, get definitions for these fuzzy matches and append them to existing preliminary_ner_results dictionary
get_definition = []

for sub_dict in fuzz_dict.values():
  for key in sub_dict.keys():
    get_definition.append(key)

get_definition = list(set(get_definition))

In [ ]:
# Get the API key and set the model name
model = "gpt-4o-mini"
client = OpenAI(api_key=userdata.get('ChatGPT'))

# Function to return answers based on prompt
def gpt_answers(topic):
  completion = client.chat.completions.create(
    model=model,
    messages=[
      {"role": "system", "content": f"""I will ask you to provide a short summary of a topic, 2 or 3 sentences long. Output must not include mathematical notation or history.
                  Example:

                  Question:
                  Markov chain

                  Answer:
                  A Markov Chain is a process where the next step depends only on the current state, not the path taken to get there. It consists of a set of states and the probabilities of transitioning from one state to another. This makes it a simple way to model systems that evolve step by step with memory only of the present.

                  End of example. """}, # system message that provides context to the model
      {"role": "user", "content": f"""Provide a short summary of: {topic}"""}  # topic query
    ]
  )

  response = completion.choices[0].message.content
  return response

In [ ]:
try:
  # Load existing responses if the file exists
  with open('fuzzy_gpt_responses.pkl', 'rb') as file:
    fuzzy_gpt_responses = pickle.load(file)
except (FileNotFoundError, EOFError):
  fuzzy_gpt_responses = dict()

# Iterate over definitions and save each response as it's created
for definition in get_definition:
  if definition not in fuzzy_gpt_responses:
    fuzzy_gpt_responses[definition] = gpt_answers(definition)
    with open('fuzzy_gpt_responses.pkl', 'wb') as file:
      pickle.dump(fuzzy_gpt_responses, file)

In [ ]:
print(f"fuzzy_gpt_responses dictionary length: {len(fuzzy_gpt_responses)}")
random_sample = random.sample(list(fuzzy_gpt_responses.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n {value}")

fuzzy_gpt_responses dictionary length: 7512
general analysis:
 General analysis refers to the systematic examination and evaluation of various components within a given subject or context. It involves breaking down complex information into manageable parts to understand relationships, patterns, and underlying principles. This approach is commonly applied in fields such as business, science, and literature to inform decision-making and enhance understanding.
triangles:
 A triangle is a three-sided polygon characterized by three edges and three vertices. The sum of the interior angles of a triangle is always 180 degrees, making it a fundamental shape in geometry. Triangles can be classified based on their sides and angles, leading to various types such as equilateral, isosceles, and scalene, as well as acute, right, and obtuse triangles.
Forbidden Induced Subgraphs:
 Forbidden induced subgraphs refer to specific subgraphs that are not allowed to appear within a larger graph when analyzin

In [ ]:
# Encode new definitions
dict_file_name = 'ner_fuzzy_encoded.pkl'

try:
  with open(dict_file_name, 'rb') as file:
    ner_fuzzy_encoded = pickle.load(file)

except FileNotFoundError:
  print(f"{dict_file_name} not found, creating {dict_file_name}...")
  start_time = time.time()

  ner_fuzzy_encoded = dict()

  for i, (key, value) in enumerate(fuzzy_gpt_responses.items()):
    if key not in ner_fuzzy_encoded:
      text_index = idx2string(value)
      tokens = tag_tokenizer(value).tokens()
      ner_results = tags(value)
      ner_fuzzy_encoded[key] = {
          "text" : value,
          "text_index": text_index,
          "text_start_end_index": idx2wordpos(value, text_index),
          "tokens" : tokens,
          "baseline_tags" : baseline_tags_list(tokens, ner_results),
          "input_ids" : tag_tokenizer.encode(value),
          }

    if i % 1000 == 0 or i == len(fuzzy_gpt_responses) - 1:
      print(f"Processed {i+1} definitions, saving to file...")
      end_time = time.time()
      elapsed_time = (end_time - start_time) / 60
      print(f"***Execution time: {elapsed_time:.2f} minutes")
      with open(dict_file_name, 'wb') as file:
        pickle.dump(ner_fuzzy_encoded, file)

In [ ]:
print(f"ner_fuzzy_encoded dictionary length: {len(ner_fuzzy_encoded)}")
random_sample = random.sample(list(ner_fuzzy_encoded.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n {value}")

ner_fuzzy_encoded dictionary length: 7512
dvoretzky's theorem:
 {'text': "Dvoretzky's theorem is a result in metric geometry that states that in every high-dimensional convex body, one can find a significant lower-dimensional section that behaves like a Euclidean ball. This implies that even as dimensions increase, these convex bodies retain properties similar to lower-dimensional spaces, providing insights into their geometric structure and behavior. The theorem has implications in various areas, including functional analysis and probability.", 'text_index': {0: 'Dvoretzky', 1: "'", 2: 's', 3: 'theorem', 4: 'is', 5: 'a', 6: 'result', 7: 'in', 8: 'metric', 9: 'geometry', 10: 'that', 11: 'states', 12: 'that', 13: 'in', 14: 'every', 15: 'high', 16: '-', 17: 'dimensional', 18: 'convex', 19: 'body', 20: ',', 21: 'one', 22: 'can', 23: 'find', 24: 'a', 25: 'significant', 26: 'lower', 27: '-', 28: 'dimensional', 29: 'section', 30: 'that', 31: 'behaves', 32: 'like', 33: 'a', 34: 'Euclidean', 3

In [ ]:
# Merge all data into single dictionary
all_data = dict()

for key_1, value_1 in ner_fuzzy_encoded.items():
  all_data.update({key_1 : value_1})

for key_2, value_2 in preliminary_ner_results.items(): #preliminary_ner_results
  if key_2 not in all_data:
    all_data.update({key_2 : value_2})

In [ ]:
print(f"all_data dictionary length: {len(all_data)}")
random_sample = random.sample(list(all_data.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n {value}")

all_data dictionary length: 11130
equivalent metrics:
 {'text': 'Equivalent metrics refer to different ways of measuring distance or similarity in a given space that yield the same topological properties. Two metrics are considered equivalent if they induce the same open sets and convergence behavior, allowing for analysis without concern for the specific metric used. This concept is important in various fields such as analysis and geometry, where different metrics can be used to explore similar phenomena.', 'text_index': {0: 'Equivalent', 1: 'metrics', 2: 'refer', 3: 'to', 4: 'different', 5: 'ways', 6: 'of', 7: 'measuring', 8: 'distance', 9: 'or', 10: 'similarity', 11: 'in', 12: 'a', 13: 'given', 14: 'space', 15: 'that', 16: 'yield', 17: 'the', 18: 'same', 19: 'topological', 20: 'properties', 21: '.', 22: 'Two', 23: 'metrics', 24: 'are', 25: 'considered', 26: 'equivalent', 27: 'if', 28: 'they', 29: 'induce', 30: 'the', 31: 'same', 32: 'open', 33: 'sets', 34: 'and', 35: 'convergence', 

In [ ]:
def find_sublist_indices(test_list, sub_list):
  n, m = len(test_list), len(sub_list)
  indices = []
  for i in range(n - m + 1):
    if test_list[i:i + m] == sub_list:
      indices.append(list(range(i, i + m)))
  return indices if indices else None  # Return None if no match is found

In [ ]:
dict_file_name = 'all_data_matches.pkl'
list_file_name = 'non_matches.pkl'

try:
  with open(dict_file_name, 'rb') as file:
    all_data = pickle.load(file)

except FileNotFoundError:
  print(f"{dict_file_name} not found, creating {dict_file_name}...")
  start_time = time.time()

  non_matches = set()

  for i, (key, sub_dict) in enumerate(all_data.items()):
    all_data_input_ids = sub_dict['input_ids']
    updated_tags = sub_dict['baseline_tags'].copy()
    # Only keep existing B, I-LOC and B, I-ORG tags
    updated_tags = ['O' if tag not in ['B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'] else tag for tag in updated_tags]
    my_list = []

    for classification_tuple, classification_ner in input_id_mapping.items():
      to_find = list(classification_tuple)
      idx_result = find_sublist_indices(all_data_input_ids, to_find)

      if idx_result:
        idx_list = [i for item in idx_result for i in item]
        my_list.append(idx_list)
        if len(idx_list) > len(classification_ner):
          classification_ner_list = list(len(idx_list)*classification_ner)
        if len(idx_list) == len(classification_ner):
          classification_ner_list = classification_ner
        if len(idx_list) < len(classification_ner):
          print(f"Error")
        for idx, classification in zip(idx_list, classification_ner):
          updated_tags[idx] = classification
      else:
        non_matches.add(key)
    ner_indices = [i for item in my_list for i in item]
    all_data[key]['ner_tags'] = updated_tags
    all_data[key]['ner_indices'] = ner_indices

    if i % 1000 == 0 or i == len(all_data) - 1:
      print(f"Processed {i+1} of {len(all_data)}, saving to file...")
      end_time = time.time()
      elapsed_time = (end_time - start_time)/60
      print(f" ***Execution time: {elapsed_time:.2f} minutes")
      with open(dict_file_name, 'wb') as file:
        pickle.dump(all_data, file)
      with open(list_file_name, 'wb') as file:
        pickle.dump(non_matches, file)

In [ ]:
print(f"all_data dictionary length: {len(all_data)}")
random_sample = random.sample(list(all_data.items()), 3)

for key, value in random_sample:
  print(f"{key}:\n {value}")

all_data dictionary length: 11130
harada-norton group:
 {'text': 'The Harada-Norton group is a notable example of a finite group in group theory, characterized as a simple group that serves as a significant example in the investigation of group classification and properties. This group has implications in various areas of mathematics, including geometry and symmetry. Its structure showcases unique features that make it particularly interesting to mathematicians studying group actions and related concepts.', 'text_index': {0: 'The', 1: 'Harada', 2: '-', 3: 'Norton', 4: 'group', 5: 'is', 6: 'a', 7: 'notable', 8: 'example', 9: 'of', 10: 'a', 11: 'finite', 12: 'group', 13: 'in', 14: 'group', 15: 'theory', 16: ',', 17: 'characterized', 18: 'as', 19: 'a', 20: 'simple', 21: 'group', 22: 'that', 23: 'serves', 24: 'as', 25: 'a', 26: 'significant', 27: 'example', 28: 'in', 29: 'the', 30: 'investigation', 31: 'of', 32: 'group', 33: 'classification', 34: 'and', 35: 'properties', 36: '.', 37: 'This

In [ ]:
for_df = ['tokens', 'input_ids', 'baseline_tags', 'ner_tags']
my_list = []
for i in for_df:
  my_list.append(pd.DataFrame(all_data[random_sample[2][0]][i], columns=[i]))

df = pd.concat(my_list, axis=1)
df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
tokens,[CLS],An,Associated,V,##ector,B,##und,##le,is,a,construction,in,differential,geometry,that,combines,a,base,manifold,with,a,vector,space,",",allowing,for,the,assignment,of,a,vector,to,each,point,of,the,manifold,in,a,way,that,respects,the,manifold,',s,structure,.,This,bundle,is,typically,associated,with,a,principal,bundle,and,provides,a,framework,for,understanding,the,interactions,between,geometric,##al,and,top,##ological,properties,.,It,plays,a,key,role,in,various,fields,",",including,physics,and,engineering,",",particularly,in,the,study,of,gauge,theories,and,fiber,bundle,##s,.,[SEP]
input_ids,101,1760,12835,159,20302,139,6775,1513,1110,170,2058,1107,12630,12053,1115,14215,170,2259,22502,1114,170,9479,2000,117,3525,1111,1103,8641,1104,170,9479,1106,1296,1553,1104,1103,22502,1107,170,1236,1115,18788,1103,22502,112,188,2401,119,1188,15119,1110,3417,2628,1114,170,3981,15119,1105,2790,170,8297,1111,4287,1103,10393,1206,16735,1348,1105,1499,7542,4625,119,1135,2399,170,2501,1648,1107,1672,3872,117,1259,7094,1105,3752,117,2521,1107,1103,2025,1104,7405,7997,1105,12753,15119,1116,119,102
baseline_tags,O,O,O,I-MISC,I-MISC,I-MISC,I-MISC,I-MISC,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O
ner_tags,O,O,B-TOPOLOGY,B-TOPOLOGY,O,I-TOPOLOGY,O,O,O,O,O,O,B-CALCULUS-AND-ANALYSIS,B-GEOMETRY,O,O,O,B-TOPOLOGY,I-TOPOLOGY,O,O,B-TOPOLOGY,B-TOPOLOGY,O,O,O,O,O,O,O,O,O,O,B-GEOMETRY,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-TOPOLOGY,O,O,O,O,O,B-TOPOLOGY,I-TOPOLOGY,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-APPLIED-MATHEMATICS,O,O,O,O,O,O,O,O,O,B-TOPOLOGY,I-TOPOLOGY,O,O,O
